# Optimization with JuMP in Julia
<table>
<tr>
<td> <a href="http://www.sloansportsconference.com"><img src="figures/ssac.jpg" alt="SSAC" style="max-height: 150px;"/> </a></td>
<td> <a href="http://julialang.org"><img src="figures/julia.png" alt="Julia" style="max-height: 150px;"/></a></td>
 <td> <a href="https://github.com/JuliaOpt/JuMP.jl"><img src="figures/jump.png" alt="IJulia" style="max-height: 150px;"/></a></td>
<td> <a href="http://jupyter.org"><img src="figures/jupyter.png" alt="Jupyter" style="max-height: 150px;"/></a></td>
</tr></table>

### *Sebastien Martin*
PhD Candidate in Operations Research, MIT Sloan. 

*Website*: **[http://sebmart.github.io](http://sebmart.github.io)**


-----------------------------------

This IJulia/Jupyter notebook is an introduction to the **Julia** language and its optimization package **JuMP**, with examples in Sports Analytics. The notebook was made for the 1-hour workshop *Optimization with JuMP in Julia* at the  [2018 MIT Sloan Sports Analytics Conference (SSAC2018)](http://www.sloansportsconference.com/), presented by Sebastien Martin.


It is distributed with OpenSource/MIT license, at [https://github.com/sebmart/optimization_jump_julia](https://github.com/sebmart/optimization_jump_julia).

The notebook features a quick overview of Julia and JuMP, with many links to more specific, thorough material. We hope readers will use this resource as a reference. It is also optimized to be presented as a slide-deck with [RISE](https://github.com/damianavila/RISE). It was built for Julia 0.6.

This notebook also includes open-source contributions from [Arthur Delarue](https://adelarue.github.io/) and [Miles Lubin](https://mlubin.github.io/).

-----------------------------------

To learn how to write Julia code and run this notebook by yourself, please refer to the Appendix 1, 2 and 3 at the end of this notebook.

____________________________________

To be able to run the code of this notebook, please install the packages:
- `Plots`
- `PlotlyJS`

and run the following code to load the packages:

In [ ]:
using Plots, DataFrames, Distributions, JuMP, Gurobi, Interact
plotlyjs()

# Section 1: Why Julia?

Quoting the [Julia website](http://julialang.org):
> Julia is a **high-level**, **high-performance** **dynamic** programming language for **technical computing**, with syntax that is familiar to users of other technical computing environments. It provides a sophisticated compiler, distributed parallel execution, numerical accuracy, and an extensive mathematical function library.

A **high-level** language:

- Easy to use and learn, with a similar syntax to Python/Matlab. 
- It is possible to do complicated computations quickly.

For example, to get the sum of all the squares of all the numbers than 1 million:

In [ ]:
sum(n^2 for n in 1:1_000_000)

A **dynamic** language:

- Julia is, like Python, Matlab or R, a dynamic language: you can interact with the language without the need to compile your code. It is easy to use it for prototyping or presenting results.

A **high-performance** language: Julia is fast. Vanilla Julia code can reach performance comparable to C and Fortran (i.e., **very fast**).
<a href="http://nbviewer.jupyter.org/url/julialang.org/benchmarks/benchmarks.ipynb"><img src="figures/Julia-benchmarks.png" alt="Julia" style="width: 1500px;"/></a>

A language for **technical computing**:
- Julia has a lot of built in functions for scientific computing.
- A growing number of packages, mostly written in Julia itself.
- Powerful editors like *Jupyter* (used for this presentation)
- More and more users in Finance, Biology, Optimization, Analytics...

> Julia is a perfect compagnon for **Sports Analytics**


In [179]:
x = y = linspace(-15,15,100)
f(x,y) = sin(sqrt(x^2+y^2))/sqrt(x^2+y^2)
plot(x,y,f,st=:surface, legend=false, size=(800,600))

# Section 2: Optimization and JuMP

Analytics allows us to make sense of Big Data, and to build strong predictive models.

But the time comes when decisions need to be made that take into account these insights and predictions, **Optimization** is the field that matters.

<img src="figures/datascience-1.svg" alt="datascience diagram part 1" style="max-height: 800px;"/>

<img src="figures/datascience-2.svg" alt="datascience diagram part 2" style="max-height: 800px;"/>

## Sports Decisions
Sports Analytics can help making a lot of important decisions
- **Scheduling** games (what teams, when)
- **Choosing** team players (also for fantasy games)
- Pitching order in baseball, substitution **strategy** in basketball...
- **Personalized** training and nutrition.

## Optimization Formulations
<img src="figures/optimization.svg" alt="decisions constraints objective" style="max-height: 800px;"/>
- **Convex/Linear Optimization**: "easy" to solve, but limited modeling power.
- **Integer Optimization**: make choices between different possibilities, hard but more powerful.

An optimization problem is defined by:
- *decision variables*, "What are the values I want to find?"
- *constraints*, "What are the rules and constraints that limit my choice of decision variables?"
- *objective*, "What is my goal, what quantity do I want to maximize/minimize?"

<img src="figures/JuMP.svg" alt="decisions" style="max-height: 800px;"/>


# Section 3: Application, *Building a team*

We discover the use of JuMP for optimization with the simple task of picking 5 players to create a team.

Creating the fake data:

In [ ]:
srand(4) # setting the random seed for reproducable results
players = DataFrame()
players[:Name] = ["Anna","Benjamin","Charlotte","Daniel","Emma","Francisco","Grace","Hunter","Isabella","Jacob","Kaylee","Liam","Mia","Noah","Olivia","Parker","Quinn","Ryan","Sophia","Thomas","Ursula","Vincent","Willow","Xavier","Yaretzi","Zachary"]
players[:Performance] = rand(Exponential(1), nrow(players))
players[:Performance] = round.(Int, players[:Performance] * 100 / maximum(players[:Performance]))
players[:Cost] = round.(Int,rand(LogNormal(0,0.2), nrow(players)).*players[:Performance]) * 10_000
players

## The data
Our Analytics team gave us estimated performance and cost of 26 players. Let's use JuMP to build the best team of 5 (the one with maximum total performance for a limited budget).


In [ ]:
players

## Decisions
First, one decision for each player : should the player be in the team? 1 = YES, 0 = NO

In [ ]:
m = Model(solver=GurobiSolver())
@variable(m, choosePlayer[1:nrow(players)], Bin)

## Constraints
- We need to choose 5 players, no more no less
- We need to respect a budget (here 1M dollars)

In [ ]:
@constraint(m, teamwith5players, sum(choosePlayer) == 5)

In [ ]:
BUDGET = 1_000_000 # budget of 1 million dollars
@constraint(m, budgetconstraint, sum(choosePlayer[p]*players[p,:Cost] for p in 1:nrow(players)) <= BUDGET)

## Objective
We want to maximize the sum of the performance of our 5 players.

In [ ]:
@objective(m, Max, sum(choosePlayer[p]*players[p,:Performance] for p in 1:nrow(players)))

Let's look at the results!

In [ ]:
solve(m)

In [ ]:
team = players[[p for p in 1:nrow(players) if getvalue(choosePlayer[p]) == 1],:]

In [ ]:
println("Total Cost: \$", sum(team[:Cost]))
println("Total Performance: ", sum(team[:Performance]))

Let's put all of this together

In [ ]:
function choosePlayers(players, budget)
    m = Model(solver=GurobiSolver(OutputFlag=0))
    @variable(m, choosePlayer[1:nrow(players)], Bin)
    @constraint(m, teamwith5players, sum(choosePlayer) == 5)
    @constraint(m, budgetconstraint, sum(choosePlayer[p]*players[p,:Cost] for p in 1:nrow(players)) <= budget)
    @objective(m, Max, sum(choosePlayer[p]*players[p,:Performance] for p in 1:nrow(players)))
    if solve(m, suppress_warnings=true) != :Optimal
        return players[Int[],:] # returns nothing if infeasible
    else
        return players[[p for p in 1:nrow(players) if getvalue(choosePlayer[p]) == 1],:]
    end 
end
budgets = 100_000*(1:50)
teams = [choosePlayers(players, budget) for budget in budgets];

Let's play with the budget!

In [ ]:
@manipulate for budget in 100_000*(1:50)
    team = deepcopy(teams[findfirst(budgets,budget)])
    push!(team, ["Total", sum(team[:Performance]), sum(team[:Cost])])
    team
end

In [ ]:
performances = [sum(team[:Performance]) for team in teams]
plot(budgets, performances, lw=3, ylims = (0,350), label="", xlabel="Budget (dollars)", ylabel="Performance")

## In practice
- This "vanilla" problem is a famous optimization problem called the **knapsack problem**.
- **More constraints** (type of players, more complex objective...) can make it useful in practice.
- Such strategies have been successfully used to win at **fantasy sports**. See [SACC2017](https://www.youtube.com/watch?v=S7DN7ThD5mM) talk by MIT professor Tauhid Zaman about fantasy baseball.

# Thank you! 

This presentation is available at [https://github.com/sebmart/optimization_jump_julia](). It contains a lot of additional material and links to learn more about Julia if you are interested.
<table>
<tr>
<td> <a href="http://www.sloansportsconference.com"><img src="figures/ssac.jpg" alt="SSAC" style="max-height: 150px;"/> </a></td>
<td> <a href="http://julialang.org"><img src="figures/julia.png" alt="Julia" style="max-height: 150px;"/></a></td>
 <td> <a href="https://github.com/JuliaOpt/JuMP.jl"><img src="figures/jump.png" alt="IJulia" style="max-height: 150px;"/></a></td>
<td> <a href="http://jupyter.org"><img src="figures/jupyter.png" alt="Jupyter" style="max-height: 150px;"/></a></td>
</tr></table>

### *Sebastien Martin*
PhD Candidate in Operations Research, MIT Sloan. 

*Website*: **[http://sebmart.github.io](http://sebmart.github.io)**

# Appendix 1: Jupyter notebooks in Julia
In this work, we use Julia through a Jupyter notebook, a useful tool (originally for Python) made available to Julia by the IJulia project.

### What is a Jupyter Notebook?
- Jupyter notebooks are **documents** (like a Word document) that can contain and run code.
- They were originally created for Python as part of the IPython project, and adapted for Julia by the **IJulia** project.
- They are very useful to **prototype**, draw **plots**, or even for teaching material like this one.
- The document relies only on a modern browser for rendering, and can easily be **shared**.

### Installing IJulia and loading this notebook
Once Julia is installed, start julia and just run the following command to install the `IJulia` package (you did this on the pre-assignment).
```jl
Pkg.install("IJulia")
```
This should work on its own. If there is any issue, check out the [IJulia website](https://github.com/JuliaLang/IJulia.jl).

Once IJulia is installed, go to the notebook file (_.ipynb_) directory, start julia and run:
```jl
using IJulia
notebook()
```
A webpage should open automatically, just click on the notebook to load it.

### Navigating the notebook

- Click `Help -> User Interface Tour` for a guided tour of the interface.
- Each notebook is composed of **cells**, that either contain code or text (`Markdown`).
- You can edit the content of a cell by double-clicking on it (_Edit Mode_).

When you are not editing a cell, you are in _Command mode_ and can edit the structure of the notebook (cells, name, options...)

- Create a cell by:
    - Clicking `Insert -> Insert Cell`
    - Pressing `a` or `b` in Command Mode
    - Pressing `Alt+Enter` in Edit Mode

- Delete a cell by:
    - Clicking `Edit -> Delete Cell`
    - Pressing `dd`

- Execute a cell by:
    - Clicking `Cell -> Run`
    - Pressing `Ctrl+Enter`

Other functions:
- Undo last text edit with `Ctrl+z` in Edit Mode
- Undo last cell manipulation with `z` in Command Mode
- Save notebook with `Ctrl+s` in Edit Mode
- Save notebook with `s` in Command Mode

Though notebooks rely on your browser to work, they do not require an internet connection (except for math rendering).

### Get comfortable with the notebook
Notebooks are designed to not be fragile. If you try to close a notebook with unsaved changes, the browser will warn you.

Try the following exercises:

>**\[Exercise\]**: Close/open

>1. Save the notebook
>2. Copy the address
>3. Close the tab
>4. Paste the address into a new tab (or re-open the last closed tab with `Ctrl+Shift+T` on Chrome)

>_The document is still there, and the Julia kernel is still alive! Nothing is lost._

>**\[Exercise\]**: Zoom

>Try changing the magnification of the web page (`Ctrl+, Ctrl-` on Chrome).

>_Text and math scale well (so do graphics if you use an SVG or PDF backend)._

>**\[Exercise\]**: MathJax
>1. Create a new cell, and select the type `Markdown` (or press `m`)
>2. Type an opening \$, your favorite mathematical expression, and a closing \$.
>3. Run the cell to render the $\LaTeX$ expression.
>4. Right-click the rendered expression.

# Appendix 2: Coding in Julia

This section is a brief introduction to Julia. It is not a comprehensive tutorial but more a _taste_ of the language for those who do not know it, and a showcase of cool features for those who already know Julia.

Very good [tutorials](http://julialang.org/learning/) are available online and in books if you are interested in learning the language.

### Basic use
Julia, as a dynamic language, can simply be used as a calculator:

In [ ]:
1+1

In [ ]:
sin(exp(2*pi)+sqrt(3))

The building blocs of Julia code are variables:

In [ ]:
a = 1
b = 2
# This is a comment 
c = a^2 + b^3 

Julia supports the common `if`, `while` and `for` structures:

In [ ]:
if c >= 10
    print("Hello")
else
    print("World")
end

In [ ]:
i = 1
while i <= 5
    println("Why, hello!") # Print with a new line
    i += 1
end

In [ ]:
for i = 1:3
    print("$i banana") # '$' can be used to insert variables into text
    if i>1
        print("s")
    end
    println() # Just a new line
end

**Do not worry about writing loops**: in Julia, they are as fast as writing vectorized code, and sometimes faster!

**Arrays** (list of numbers) are at the core of research computing and Julia's arrays are extremely optimized.

In [ ]:
myList = [1, 2, 3]

Array indexing starts with 1 in Julia:

In [ ]:
myList[1]

In [ ]:
myList[3] = 4 
myList

A 2-dimensional array is a Matrix

In [ ]:
A = [1 2 3
     2 1 2
     3 2 1]

A = [1 2 3; 2 1 2; 3 2 1] #same thing

Matrices can be multiplied, inverted...

In [ ]:
A^-1 #inverse

A^2 * A^-1

In [ ]:
A*[1,2,3]

In [ ]:
eigenValues, eigenVectors = eig(A)
eigenValues

Let's try a quick exercise!

> **[Exercise]** A random variable $X\sim\text{Bin}(n,p)$ is defined as the number of successes in $n$ trials where each trial has a success probability $p$. For example, if $X=\text{Bin}(10,0.5)$, then $X$ is the number of coin flips that turn up heads in 10 flips of a fair coin.

> Using only the function `rand()`, which generates a uniformly random number between 0 and 1, write a function `binomial_rv(n,p)` that outputs a single draw of a binomial random variable with parameters `n` and `p`.

In [ ]:
function binomial_rv(n, p)
    # your code here
    
end
binomial_rv(10,0.5)

### Just-in-time compilation

We mentioned earlier that one of the reasons Julia is fast is _just-in-time_ compilation. This means that right before a function is executed, Julia compiles it and optimizes it. Function compilations are also cached for future use.

In [ ]:
function countTo(n)
    count = 0
    @time for i = 1:n
        count += 1
    end
    return count
end
println("First use: slow like a dynamic language")
@time countTo(10_000_000) 
println("Second use: compiled and optimized automatically")
@time countTo(10_000_000);

### Type stability

Other interpreted languages have just-in-time compilers (e.g. Python). Why is Julia better?

**Types:** Everything has a type in Julia

In [ ]:
typeof(1)

In [ ]:
typeof(1.5)

In [ ]:
typeof("abc")

Type stability is the idea that there is only one possible type which can be output by a method. For example, the reasonable type to output from `*(::Int64,::Int64)` is an `Int64`. No matter what you give it, it will spit out an `Int64`.

This is called **[multiple dispatch](https://en.wikipedia.org/wiki/Multiple_dispatch)** : the `*` operator calls a different method depending on the types that it sees.

In [ ]:
1//2 # fraction in Julia

In [ ]:
typeof(1//2)

In [ ]:
(1//2)*(1//2)

In [ ]:
(0.5)*(0.5) # The same function gives different results depending on the type

In [ ]:
(im)*(im) # This also works with complex numbers

In [ ]:
function myFunction(x)
    println("Default output")
end

function myFunction(x::Int) # only called when x is an integer
    println("You gave me an integer!")
end

myFunction(1.0)
myFunction(1)
myFunction("ORC")

#### How does type stability help?

To explore the power of Julia's type-stable system, we will use code introspection macros to see what the code actually compiles to. Let's look at the code in LLVM, a portable assembly language:

In [ ]:
@code_llvm 2*5

The code above is short and sweet: it multiplies the two numbers and returns the result. It turns out that the compiled code is _the same_ as the compiled version of the same code written in C or Fortran.

_Question_: in what cases does the code compile to something as efficient as C/Fortran?

_Answer_: **type-stability**. If a function is type-stable, then the compiler can know what the type will be at all points in the function and smartly optimize it to the same assembly as C/Fortran. If it is not type-stable, Julia has to add expensive "boxing" to ensure types are found/known before operations are performed.

Requiring type stability as a design decision has some interesting effects. Let's try to raise an integer to a negative power.

In [ ]:
2^(-5)

In any other interpreted language, the above command would work and return the answer as a `Float64`. In Julia, `^(Int64, Int64)` must return an `Int64`, which cannot represent $2^{-5}=0.03125$.

#### What is the price of type instability?

Let's write a function that can raise integers to negative exponents, and test that it works.

In [ ]:
function expo(x,y)
    if y>0
        return x^y
    else
        x = convert(Float64,x)
        return x^y
    end
end
@assert expo(2, 5) == 32

Let's inspect the type-stable exponentiation code in LLVM.

In [ ]:
@code_llvm ^(2,5)

Now let's inspect the type-unstable code we wrote:

In [ ]:
@code_llvm expo(2,5)

A lot more functionalities are available and for you to discover!

In [ ]:
l = [i^2 for i in 1:10 if i%2 == 0] # list comprehensions (similar to Python)

In [ ]:
sumEvenSquares = sum(i^2 for i in 1:10 if i%2 == 0) # summing over an iterator

### Navigating Julia
Julia has a package manager to quickly download, install, update and uninstall new tools (_packages_)

In [ ]:
# Add Packages Plots, and Pyplot (can take some time)
Pkg.add("Plots")
Pkg.add("PyPlot")
# Update
Pkg.update()
#Remove:
# Pkg.rm("PyPlot")

Use `?` to get the documentation of a function

In [ ]:
?eig

Use tab-completion to auto-complete functions and variables names: try ``myF<TAB>``:

In [ ]:
myF
fac

The ``methods`` function lists all of the different implementations of a function depending on the input types.
Click on the link to see the Julia source code.

In [ ]:
methods(sin)

### Plotting in IJulia
There are several Julia plotting packages. 
- [Plots][4] is a _meta_ plotting package, that can use any other plotting package to make the same plot (including all of the following).
- [Plotly supports Julia][2], for great interactive visualization with the `PlotlyJS` package.
- [PyPlot.jl][3] is a Julia interface to Matplotlib, and should feel familiar to both MATLAB and Python users.
- [Gadfly][1] is written entirely in Julia, inspired by ggplot2, and concentrates on statistical graphics.
- And a lot more

Jupyter/IJulia will render the plots directly on the notebook!

[1]: https://github.com/dcjones/Gadfly.jl
[2]: https://plot.ly/julia/
[3]: https://github.com/stevengj/PyPlot.jl
[4]: https://juliaplots.github.io

In [ ]:
using Plots
pyplot() #plot using Matplotlib
x = linspace(0,5,1000)
plot(x, sin.(x.^2))

Note that the first plot while always take a few seconds to be drawn, a consequence of Julia's just in time compilation. Lots of other plot types are available

# Appendix 3: Advanced use of Julia and Notebooks
The following is just a sample of what can be done with Julia and notebooks. Feel free to explore by yourself any item of interest

### Notebooks
Jupyter notebooks have a lot of interesting hidden functionalities!

**Github and sharing**

If you save your .ipynb notebook file in a .git project, hosted on Github, you can easily visualize and share it online (in non-interactive mode).

For example, this notebook is available at https://github.com/sebmart/intro-julia-jupyter/blob/master/intro-julia-jupyter.ipynb

You can also use [Gist](https://gist.github.com) and [nbviewer](http://nbviewer.jupyter.org) to quickly share a notebook (for example to your advisor) without creating a git repo.

**Converting your notebook**

Jupyter notebooks are a popular format that can be converted to a variety of types of documents, depending on your needs:
- Latex
- HTML
- PDF
- Slides with Reveal.JS (used to present this notebook!)
- Markdown ...

These conversions use the [`nbconvert`](https://github.com/jupyter/nbconvert) command.

**Remote computing**

The Notebook system is a web interface. Notebooks can be run on another computer. This is useful if you want your code to run on a more powerful remote machine.

[Port-forwarding](https://help.ubuntu.com/community/SSH/OpenSSH/PortForwarding) through SSH is a good start for this.

**Advanced Markdown**

Jupyter text cells use Markdown for formatting. Markdown is an easy to use formatting language (a little like HTML or LaTeX in more simple). You can use the text of this notebook as an example, or learn more [here](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet).

Jupyter uses _Github flavored Markdown_, and is particularly good at displaying math and colored code. You can even include a video!

### Julia
We only presented a small subset of Julia functionalities. We list here of few interesting things you may not know.

**Using the command line from Julia**

You can run bash commands directly from Julia by starting the command with a semicolon: `;`

In [ ]:
;ls

**Unicode character** 

You can use unicode characters as part of variables and function names in Julia. You can use $\LaTeX$-style autocompletes in the Julia terminal or a Jupyter/IJulia code-cell to write them. Some of them are already defined Julia constants and functions
> Try to type `\pi<TAB>` in a cell.

In [ ]:
π

In [ ]:
"carrot" ∈ ["potato", "tomato", "carrot"]

**Juno**

Julia has a very nice and powerful text editor, [_Juno_](http://junolab.org), that is built on [Atom](https://atom.io). It is very similar to the Matlab interface or RStudio. Functionalities include:
- Autocomplete
- Integrated Plotting
- Debugging, Manual, ...

It is better suited for serious projects with several files, when an IJulia notebook is not enough.

**Advanced Julia functionalities**
Julia is a state-of-the-art programming language, with lots of useful functionalities, including:
- [Powerful Macros](http://docs.julialang.org/en/release-0.6/manual/metaprogramming/) (meta-programming)
- [Code testing](http://docs.julialang.org/en/release-0.6/stdlib/test/)
- [User-defined types](docs.julialang.org/en/release-0.6/manual/types/), that are as fast as built-in ones.
- [Package creation](http://docs.julialang.org/en/release-0.6/manual/modules/)
- A new [debugger](https://github.com/Keno/Gallium.jl) 

### Interesting Packages
Using the Package eco-system, there is almost nothing you cannot achieve:

-       Advanced Plotting   with [**Plots.jl**](https://juliaplots.github.io). Functionalities include 3D-plots, animated plots, stats plots, home-made plot "recipes" ...

- Call any python call using [**PyCall.jl**](https://github.com/JuliaPy/PyCall.jl). (you can also interacts with several other languages)

- Advanced graphs/networks algorithms with [**LightGraphs.jl**](https://github.com/JuliaGraphs/LightGraphs.jl)

- Applications in [**Finance**](https://github.com/JuliaQuant), [**Biology**](https://github.com/BioJulia/Bio.jl), [**Stats and Machine Learning**](http://juliastats.github.io), [**Optimization**](http://www.juliaopt.org) (including the great [**JuMP**](https://github.com/JuliaOpt/JuMP.jl) package!)

- Save and load your variables or environment to a file with [**JLD.jl**](https://github.com/JuliaIO/JLD.jl)

- More data structures in [**DataStructures.jl**](https://github.com/JuliaLang/DataStructures.jl)

- And a lot more in the **over 1200 registered packages**!